In [ ]:
import os
import numpy as np

In [ ]:
param_labels = ['pich_mean',       'yaw_var',       'volume_mean',       'tone_var']
score_labels = ['pich_mean_score', 'yaw_var_score', 'volume_mean_score', 'tone_var_score']

In [ ]:
result_files = [filename for filename in os.listdir('results') if not filename == '.gitkeep']
scores_list = np.array([], dtype='int32')
param_table = {label : np.array([], dtype="float32") for label in param_labels}

for file_name in result_files:
    base_name, _, _ = file_name.split('.')
    scores = base_name.split('_')
    scores_list = np.append(scores_list, [[int(score) for score in scores]])

    with open(os.path.join('./results', file_name)) as f:
        for line in f.read().splitlines():
            label, value = line.split(':')
            param_table[label] = np.append(param_table[label], float(value))

scores_list  = scores_list.reshape(-1, 4)
scores_table = {label : scores for label, scores in zip(score_labels, scores_list.T)}

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge

In [ ]:
model_table = {label : None for label in score_labels}

for (X_label, X), (y_label, y) in zip(param_table.items(), scores_table.items()):
    X, y = X.reshape(-1, 1), y.reshape(-1, 1)
    # train
    model_table[y_label] = Ridge()
    model_table[y_label].fit(X, y)
    
    # predict
    p = model_table[y_label].predict(X)
    print("Score: ", model_table[y_label].score(X, y))
    
    # plot
    plt.figure()
    plt.xlabel(X_label)
    plt.ylabel(y_label)
    plt.scatter(X, y)
    plt.plot(X, p)
    plt.show()

In [ ]:
import pickle

for label, model in model_table.items():
    filename = label+".sklean"
    pickle.dump(model, open(os.path.join('data', filename), 'wb'))